In [16]:
pip install cv2module

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip  install colormath

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for colormath: filename=colormath-3.0.0-py3-none-any.whl size=39408 sha256=287f32dbb69d8e09e4df02cac473d83baa13adb0eb29a9e59f3da73c20838d9b
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\41\fc\b0\725cea6b023e17948a084b1111b4941299859d468c4407e01d
Successfully built colormath
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install colour

Note: you may need to restart the kernel to use updated packages.


In [1]:
#Generic Genetic algprithm with elitism
import string
import random
from skimage.metrics import peak_signal_noise_ratio as psns # For image similarity evaluation
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np
import imageio # For gif saving

#Load and show original image for tracking (convert to black and white)
original_image = Image.open("cherry.jpg").convert("L")
original_height, original_width  = original_image.size
cv2.imshow("Original.jpg", np.array(original_image))
cv2.imwrite("Original.jpg",np.array(original_image))

#Adjust hyperparameters
NUMBER_OF_GENERATIONS = 7000
POPULATION_NUMBER = 50  # How many images in 1 generation (without elitism)
MUTATION_CHANCE = 0.1  # Chance of mutating (adding random shapes)
MUTATION_STRENGTH = 1  # How many shapes to add in mutation
ELITISM = True  # Turn on/off elitism (transfering best images to next generation without crossover)
ELITISM_NUMBER = 4  # How many best images transfer to next generation (elitism)
STARTING_SHAPE_NUMBER = 6  # How many shapes to draw on each image in first generation

PRINT_EVERY_GEN = 25  # Print fitness value every x generations
SAVE_FRAME_FOR_GIF_EVERY = 100  # Save best image every x generations for gif creation

# What to draw functions

def draw_text(image, size=20):
    """Draw random text on image with given size."""
    font = ImageFont.truetype("arial.ttf", size)
    text_length = random.randint(1,3)
    text = "".join(random.choice(string.ascii_letters) for i in range(text_length))

    x = random.randint(0,original_width-1)
    y = random.randint(0,original_height-1)

    color = (random.randint(0,255))
    image.text((y,x), text, fill=color, font=font)


def add_random_shape_to_image(image, number_of_shapes):
    """Add shape with random proporties to image number_of_shapes times."""
    image_filled = image.copy()
    for _ in range(0, number_of_shapes):
        draw = ImageDraw.Draw(image_filled)
        draw_text(draw)
    return image_filled


def create_random_population(size):
    """Create first generation with random population."""
    first_population = []
    for _ in range(0, size):
        blank_image = Image.new("L", (original_height, original_width))
        filled_image = add_random_shape_to_image(blank_image, MUTATION_STRENGTH)
        first_population.append(filled_image)
    return first_population


def evaluate_fitness(image):
    """Evaluate similarity of image with original."""
    return psns(np.array(image), np.array(original_image))


# Crossover operations with alternatives and helpers

def images_to_arrays(image1, image2):
    """Represent images as arrays."""
    img1_arr = np.array(image1)
    img2_arr = np.array(image2)
    return img1_arr ,img2_arr


def blending(image1, image2):
    """Blend to images together with 0.5 alpha."""
    return Image.blend(image1, image2, alpha=0.5)


def random_horizontal_swap(image1, image2):
    """Swap random rows of two images."""
    img1_arr, img2_arr = images_to_arrays(image1, image2)
    horizontal_random_choice = np.random.choice(original_width,
                                                int(original_width/2),
                                                replace=False)
    img1_arr[horizontal_random_choice] = img2_arr[horizontal_random_choice]
    return Image.fromarray(img1_arr)

def crossover(image1, image2):
    """Make crossover operation on two images."""
    return random_horizontal_swap(image1, image2)


def mutate(image, number_of_times):
    """Mutate image adding random shape number_of_times."""
    mutated = add_random_shape_to_image(image, number_of_times)
    return mutated


def get_parents(local_population, local_fitnesses):
    """Connect parents in pairs based on fitnesses as weights using softmax."""
    fitness_sum = sum(np.exp(local_fitnesses))
    fitness_normalized = np.exp(local_fitnesses) / fitness_sum
    local_parents_list = []
    for _ in range(0, len(local_population)):
        parents = random.choices(local_population, weights=fitness_normalized, k=2)
        local_parents_list.append(parents)
    return local_parents_list

def whole_pipeline():
    """Go through whole pipeline and execute it."""
    save_gif = [] #Creating empty frames list for gif saving at the end

    # Create first generation
    population = create_random_population(POPULATION_NUMBER)

    # Loop through generations
    for generation in range(0, NUMBER_OF_GENERATIONS):

        # Calculate similarity of each image in population to original image
        fitnesses = []
        for img in population:
            actual_fitness = evaluate_fitness(img)
            fitnesses.append(actual_fitness)

        # Get ids of best images in population
        top_population_ids = np.argsort(fitnesses)[-ELITISM_NUMBER:]

        # Start creating new population for next generation
        new_population = []

        # Connect parent into pairs
        parents_list = get_parents(population, fitnesses)

        # Create childs
        for i in range(0, POPULATION_NUMBER):
            new_img = crossover(parents_list[i][0], parents_list[i][1])
            #Mutate
            if random.uniform(0.0, 1.0) < MUTATION_CHANCE:
                new_img = mutate(new_img, MUTATION_STRENGTH)
            new_population.append(new_img)

        # Elitism transfer
        if ELITISM:
            for ids in top_population_ids:
                new_population.append(population[ids])

        # Print info every x generations
        if generation % PRINT_EVERY_GEN == 0:
            print(generation)
            print(fitnesses[top_population_ids[0]])

        # Get best actual image and show it
        open_cv_image = np.array(population[top_population_ids[0]])
        cv2.imshow("test", open_cv_image)

        # Gif creation
        if generation % SAVE_FRAME_FOR_GIF_EVERY == 0:
            save_gif.append(open_cv_image)

        cv2.waitKey(1)
        population = new_population

    # Save gif and best output
    imageio.mimsave("cherry Traditional(7000)output_gif.gif", save_gif)
    cv2.imwrite("cherry Traditional(7000)output_best.jpg", open_cv_image) 

if __name__ == "__main__":
    whole_pipeline()

0
0.7551139454698625
25
0.7874588301922815
50
0.8208348216695383
75
0.8680419391125783
100
0.9171404820735917
125
0.9582348516106204
150
1.0018505212549815
175
1.0523669851990771
200
1.0971727184271496
225
1.1404448259240678
250
1.192277299565984
275
1.233773077132513
300
1.2909350934337314
325
1.3383759355837512
350
1.387633333106177
375
1.4288677893738027
400
1.487535741763344
425
1.5369578476349535
450
1.5808967145636579
475
1.6295196226545712
500
1.6790158773127555
525
1.7211275855102224
550
1.7711452803510306
575
1.8094931683341662
600
1.8484518784231183
625
1.9033371701188284
650
1.9418512990949577
675
1.9893329017451056
700
2.040273619149529
725
2.089957441985551
750
2.150291220482572
775
2.1977811531011704
800
2.2438845369993605
825
2.285059239239427
850
2.338792324567909
875
2.375485526427232
900
2.4269081976705893
925
2.4716585126670916
950
2.520627776769629
975
2.5757122475454093
1000
2.617462461996127
1025
2.65942555121837
1050
2.7017341835451196
1075
2.755485572636129
1100

In [5]:
pip install gari

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pygad

     -------------------------------------- 56.3/56.3 kB 591.1 kB/s eta 0:00:00


In [4]:
#Genetic algorithm using transformation and elitism
import string
import random
from skimage.metrics import peak_signal_noise_ratio as psns # For image similarity evaluation
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np
import imageio # For gif saving

#Load and show original image for tracking (convert to black and white)
original_image = Image.open("Homer Simpson image.jpg").convert("L")
original_height, original_width  = original_image.size
cv2.imshow("Original.jpg", np.array(original_image))
cv2.imwrite("Original.jpg",np.array(original_image))

#Adjust hyperparameters
NUMBER_OF_GENERATIONS = 7000
POPULATION_NUMBER = 50  # How many images in 1 generation (without elitism)
MUTATION_CHANCE = 0.1  # Chance of mutating (adding random shapes)
MUTATION_STRENGTH = 1  # How many shapes to add in mutation
ELITISM = True  # Turn on/off elitism (transfering best images to next generation without crossover)
ELITISM_NUMBER = 4  # How many best images transfer to next generation (elitism)
STARTING_SHAPE_NUMBER = 6  # How many shapes to draw on each image in first generation

PRINT_EVERY_GEN = 25  # Print fitness value every x generations
SAVE_FRAME_FOR_GIF_EVERY = 100  # Save best image every x generations for gif creation

# What to draw functions

def draw_rectangle(image, size=10):
    """Draw rectangle on image with given size."""
    x = random.randint(0,original_width-1)
    y = random.randint(0,original_height-1)

    color = (random.randint(0,255))

    image.rectangle([(y,x), (y+size,x+size)], fill=color)


def draw_text(image, size=20):
    """Draw random text on image with given size."""
    font = ImageFont.truetype("arial.ttf", size)
    text_length = random.randint(1,3)
    text = "".join(random.choice(string.ascii_letters) for i in range(text_length))

    x = random.randint(0,original_width-1)
    y = random.randint(0,original_height-1)

    color = (random.randint(0,255))
    image.text((y,x), text, fill=color, font=font)


def add_random_shape_to_image(image, number_of_shapes):
    """Add shape with random proporties to image number_of_shapes times."""
    image_filled = image.copy()
    for _ in range(0, number_of_shapes):
        draw = ImageDraw.Draw(image_filled)
        draw_text(draw)
    return image_filled

def add_random_shape_to_image2(image, number_of_shapes):
    """Add shape with random proporties to image number_of_shapes times."""
    image_filled2 = image.copy()
    draw = ImageDraw.Draw(image_filled2)
    draw_rectangle(draw)
    return image_filled2

def add_random_shape_to_image3(image, number_of_shapes):
    """Add shape with random proporties to image number_of_shapes times."""
    image_filled3 = image.copy()
    draw = ImageDraw.Draw(image_filled3)
    draw_rectangle(draw)
    return image_filled3

def create_random_population(size):
    """Create first generation with random population."""
    first_population = []
    for _ in range(0, size):
        blank_image = Image.new("L", (original_height, original_width))
        filled_image = add_random_shape_to_image(blank_image, MUTATION_STRENGTH)
        first_population.append(filled_image)
    return first_population

def create_outside_gene():
    """Create outside gene for a generation."""
    first_population2 = []
    blank_image = Image.new("L", (original_height, original_width))
    filled_image = add_random_shape_to_image2(blank_image, MUTATION_STRENGTH)
    first_population2.append(filled_image)
    return first_population2

def create_new_outside_gene():
    """Create otside gene for a generation."""
    first_population3 = []
    blank_image = Image.new("L", (original_height, original_width))
    filled_image = add_random_shape_to_image3(blank_image, MUTATION_STRENGTH)
    first_population3.append(filled_image)
    return first_population3

def evaluate_fitness(image):
    """Evaluate similarity of image with original."""
    return psns(np.array(image), np.array(original_image))


# Crossover operations with alternatives and helpers

def images_to_arrays(image1, image2):
    """Represent images as arrays."""
    img1_arr = np.array(image1)
    img2_arr = np.array(image2)
    return img1_arr ,img2_arr


def blending(image1, image2):
    """Blend to images together with 0.5 alpha."""
    return Image.blend(image1, image2, alpha=0.5)


def random_horizontal_swap(image1, image2):
    """Swap random rows of two images."""
    img1_arr, img2_arr = images_to_arrays(image1, image2)
    horizontal_random_choice = np.random.choice(original_width,
                                                int(original_width/2),
                                                replace=False)
    img1_arr[horizontal_random_choice] = img2_arr[horizontal_random_choice]
    return Image.fromarray(img1_arr)

#Below can be used for crossover as well
def transformation(image1, image2):
    """Perform Transformation operation on parent and outside gene."""
    return random_horizontal_swap(image1, image2)

def mutate(image, number_of_times):
    """Mutate image adding random shape number_of_times."""
    mutated = add_random_shape_to_image(image, number_of_times)
    return mutated


def get_parent(local_population, local_fitnesses):
    """Select parent for transformation"""
    fitness_sum = sum(np.exp(local_fitnesses))
    fitness_normalized = np.exp(local_fitnesses) / fitness_sum
    local_parents_list = []
    for _ in range(0, len(local_population)):
        parents = random.choices(local_population, weights=fitness_normalized, k=1)
        local_parents_list.append(parents)
    return local_parents_list
        
def whole_pipeline():
    """Go through whole pipeline and execute it."""
    save_gif = [] # Creating empty frames list for gif saving at the end
    
    # Create Outside gene
    image2 = create_outside_gene()
    
    # Create first generation
    population = create_random_population(POPULATION_NUMBER)

    # Loop through generations
    for generation in range(0, NUMBER_OF_GENERATIONS):
        

        # Calculate similarity of each image in population to original image
        fitnesses = []
        for img in population:
            actual_fitness = evaluate_fitness(img)
            fitnesses.append(actual_fitness)

        # Get ids of best images in population
        top_population_ids = np.argsort(fitnesses)[-ELITISM_NUMBER:]
        

        # Start creating new population for next generation
        new_population = []

        # Connect parent into pairs
        parent_list = get_parent(population, fitnesses)

        # Create childs
        for i in range(0, POPULATION_NUMBER):
            new_img = transformation(parent_list[i][0], parent_list[i][0])
            #Mutate
            if random.uniform(0.0, 1.0) < MUTATION_CHANCE:
                new_img = mutate(new_img, MUTATION_STRENGTH)
            new_population.append(new_img)
        
        # Elitism transfer
        if ELITISM:
            for ids in top_population_ids:
                new_population.append(population[ids])


        # Print info every x generations
        if generation % PRINT_EVERY_GEN == 0:
            print(generation)
            print(fitnesses[top_population_ids[0]])

        # Get best actual image and show it
        open_cv_image = np.array(population[top_population_ids[0]])
        cv2.imshow("test", open_cv_image)

        # Gif creation
        if generation % SAVE_FRAME_FOR_GIF_EVERY == 0:
            save_gif.append(open_cv_image)

        cv2.waitKey(1)
        population = new_population
        image2 = create_new_outside_gene()

    # Save gif and best output
    imageio.mimsave("cherry transformation_gif(7000).gif", save_gif)
    cv2.imwrite("cherry transformation_best(7000).jpg", open_cv_image) 

if __name__ == "__main__":
    whole_pipeline()

0
1.3560533501614083
25
1.452795667345835
50
1.536354621721545
75
1.6334281829680006
100
1.7354217958067353
125
1.8698123862577436
150
2.034812445232795
175
2.131061973035853
200
2.230493227057154
225
2.3222971039606874
250
2.408431028388181
275
2.489877001290401
300
2.6068056979495036
325
2.747988740172605
350
2.832774689801573
375
2.930489908553918
400
2.997952477030565
425
3.094662816351975
450
3.176541558150652
475
3.2566716619394054
500
3.348394497154021
525
3.4320540852608126
550
3.5082918109344363
575
3.5858403817119
600
3.679182962440983
625
3.7714058666520267
650
3.8418170805179184
675
3.9358430012005923
700
4.017995047666948
725
4.089325374882785
750
4.184083831092743
775
4.259288308306623
800
4.346919646223252
825
4.419830316166161
850
4.509701115522306
875
4.583662348877786
900
4.676322154701943
925
4.726028035114828
950
4.780545194411091
975
4.825941821167944
1000
4.908555784407166
1025
5.03264820459364
1050
5.115934920381466
1075
5.191659224278196
1100
5.279935709018094
1

In [1]:
import PIL
from PIL import Image
  
# loading the image
img = PIL.Image.open("Homer Simpson image.jpg")
  
# fetching the dimensions
wid, hgt = img.size
  
# displaying the dimensions
print(str(wid) + "x" + str(hgt))

123x274


In [2]:
from PIL import Image, ImageDraw
img = PIL.Image.open("Homer simpson paper result.jpg")
img = img.resize((123,274))
img.show()
wid, hgt = img.size
print(str(wid) + "x" + str(hgt))
img = img.save("Homer simpson paper result resized.jpg")

123x274


In [3]:
import numpy as np
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psns
original_image = Image.open("Homer Simpson image.jpg").convert("L")
image = Image.open("Homer simpson paper result resized.jpg").convert("L")
value = psns(np.array(original_image), np.array(image))
print(value)

11.499129516707889
